In [1]:
import requests
import pandas as pd
import json
from dateutil import parser

In [2]:
API_KEY="bae3cb38eeca73a07ad21655fb480f75-5523673667f28c74c2239a9a82ebf23e"
ACCOUNT_ID="101-012-23708512-001"
OANDA_URL="https://api-fxpractice.oanda.com/v3/"

In [3]:
session = requests.Session()

In [4]:
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type" : "application/json"
})

In [5]:
params = dict(
count = 10,
granularity = "H1",
price = "MBA"
)

In [6]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [7]:
response = session.get(url, params=None, data=None, headers=None)

In [8]:
response.status_code

200

In [9]:
data=response.json()

In [10]:
instruments_list=data['instruments']

In [11]:
len(instruments_list)

128

In [12]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'minimumGuaranteedStopLossDistance', 'guaranteedStopLossOrderExecutionPremium', 'guaranteedStopLossOrderLevelRestriction', 'tags', 'financing'])

In [13]:
key_i=['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize','marginRate']


In [14]:
#data

In [15]:
instruments_dict={}
for i in instruments_list:
  #print(i['name'])
  key = i['name']
  instruments_dict[key]= {k:i[k] for k in key_i}

In [16]:
#instruments_dict
#instruments_dict['EUR_USD']

In [17]:
with open ("../data/instruments.json","w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [23]:
def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
    count = count,
    granularity = granularity,
    price = "MBA"
    )
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()
    
    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    return response.status_code, data


def get_candles_df(data):
    if len(data)==0:
        return pd.DataFrame()
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l','c']
    final_data = []
    for candle in data:
        if candle['complete']==False:
            continue
        new_dict = {}
        new_dict['time']= parser.parse(candle['time'])
        new_dict['volume']= candle['volume']
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"]=float(candle[p][o])
        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)
    return df


def create_data_file(pair_name, count=10, granularity="H1"):
    code, data = fetch_candles("EUR_USD",count=count, granularity=granularity)
    if code != 200:
        print("Failed", pair_name, data)
        return

    if len(data) == 0:
        print("No candles", pair_name)

    candles_df = get_candles_df(data)
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
    print(f"{pair_name} {granularity} {candles_df.shape[0]} candles, {candles_df.time.min()} {candles_df.time.max()}")
    
    

In [24]:
code, data = fetch_candles("EUR_USD",count=10, granularity="H4")
candles_df = get_candles_df(data)

In [26]:
#candles_df
create_data_file("EUR_USD",count=10, granularity="H4")

EUR_USD H4 10 candles, 2022-12-01 06:00:00+00:00 2022-12-02 18:00:00+00:00


In [29]:
#candles_df.info()
our_curr = ['EUR', 'USD','GBP', 'JPY', 'CHF', 'NZD', 'CAD','AUD']
#instruments_dict

{'NZD_CAD': {'name': 'NZD_CAD',
  'type': 'CURRENCY',
  'displayName': 'NZD/CAD',
  'pipLocation': -4,
  'displayPrecision': 5,
  'tradeUnitsPrecision': 0,
  'minimumTradeSize': '1',
  'marginRate': '0.05'},
 'EUR_SGD': {'name': 'EUR_SGD',
  'type': 'CURRENCY',
  'displayName': 'EUR/SGD',
  'pipLocation': -4,
  'displayPrecision': 5,
  'tradeUnitsPrecision': 0,
  'minimumTradeSize': '1',
  'marginRate': '0.05'},
 'EU50_EUR': {'name': 'EU50_EUR',
  'type': 'CFD',
  'displayName': 'Europe 50',
  'pipLocation': 0,
  'displayPrecision': 1,
  'tradeUnitsPrecision': 1,
  'minimumTradeSize': '0.1',
  'marginRate': '0.05'},
 'XAU_CAD': {'name': 'XAU_CAD',
  'type': 'METAL',
  'displayName': 'Gold/CAD',
  'pipLocation': -2,
  'displayPrecision': 3,
  'tradeUnitsPrecision': 0,
  'minimumTradeSize': '1',
  'marginRate': '0.05'},
 'EUR_AUD': {'name': 'EUR_AUD',
  'type': 'CURRENCY',
  'displayName': 'EUR/AUD',
  'pipLocation': -4,
  'displayPrecision': 5,
  'tradeUnitsPrecision': 0,
  'minimumTrad

In [30]:
for p1 in our_curr:
    for p2 in our_curr:
        pr = f"{p1}_{p2}"
        if pr in instruments_dict:
            for g in ["H1","H4"]:
                create_data_file(pr, count=4001, granularity=g)

EUR_USD H1 4001 candles, 2022-04-13 23:00:00+00:00 2022-12-02 21:00:00+00:00
EUR_USD H4 4001 candles, 2020-05-12 01:00:00+00:00 2022-12-02 18:00:00+00:00
EUR_GBP H1 4001 candles, 2022-04-13 23:00:00+00:00 2022-12-02 21:00:00+00:00
EUR_GBP H4 4001 candles, 2020-05-12 01:00:00+00:00 2022-12-02 18:00:00+00:00
EUR_JPY H1 4001 candles, 2022-04-13 23:00:00+00:00 2022-12-02 21:00:00+00:00
EUR_JPY H4 4001 candles, 2020-05-12 01:00:00+00:00 2022-12-02 18:00:00+00:00
EUR_CHF H1 4001 candles, 2022-04-13 23:00:00+00:00 2022-12-02 21:00:00+00:00
EUR_CHF H4 4001 candles, 2020-05-12 01:00:00+00:00 2022-12-02 18:00:00+00:00
EUR_NZD H1 4001 candles, 2022-04-13 23:00:00+00:00 2022-12-02 21:00:00+00:00
EUR_NZD H4 4001 candles, 2020-05-12 01:00:00+00:00 2022-12-02 18:00:00+00:00
EUR_CAD H1 4001 candles, 2022-04-13 23:00:00+00:00 2022-12-02 21:00:00+00:00
EUR_CAD H4 4001 candles, 2020-05-12 01:00:00+00:00 2022-12-02 18:00:00+00:00
EUR_AUD H1 4001 candles, 2022-04-13 23:00:00+00:00 2022-12-02 21:00:00+00:00